In [1]:
import numpy as np
import pandas as pd
import bs4
import requests
import datetime
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
import time
import bs4
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

In [4]:
def get_last_page(LINK):

    options = webdriver.ChromeOptions()
    # options.add_experimental_option("prefs", {"profile.managed_default_content_settings.images": 2,})
    # options.add_argument('--disk-cache-dir=/path/to/cache')
    # options.add_argument('--headless') 
    driver = webdriver.Chrome(options=options)
    driver.get(LINK)

    soup = bs4.BeautifulSoup(driver.page_source, 'html.parser')
    list_aux = [i.text for i in soup.findAll('li',{'class': 'paginate_button'})]

    pages = []
    for i in list_aux:
        try:
            pages.append(int(i))
        except:
            pass

    last_page = pages[-1]

    driver.quit()

    return last_page

def get_anuncios_links(last_page):

    anuncios_links = []

    for page in range(1,3):
        LINK = f'https://cilar.com.br/alugar/apartamento/curitiba-pr?valueType=price&valueMin=100.00&valueMax=30000.00&areaType=area_total&areaMin=0&areaMax=10000&order=1&page={page}'

        options = webdriver.ChromeOptions()
        # options.add_experimental_option("prefs", {"profile.managed_default_content_settings.images": 2,})
        # options.add_argument('--headless') 
        driver = webdriver.Chrome(options=options)
        driver.get(LINK)

        soup = bs4.BeautifulSoup(driver.page_source, 'html.parser')
        anuncios = soup.findAll('div', {'class':'box'})
        for anuncio in anuncios:
            anuncios_links.append('https://cilar.com.br' + anuncio.findAll('a')[0]['href'])

    return pd.DataFrame(anuncios_links)

def get_info_anuncio(driver, link):
    
    driver.get(link)
    soup = bs4.BeautifulSoup(driver.page_source, 'html.parser')

    anuncio_info_aux = {}

    try:
        # titulo
        anuncio_info_aux['titulo'] = soup.findAll('h1', {'class':'title title-default'})[0].text
    except:
        anuncio_info_aux['titulo'] = np.nan
    
    try:
        # titulo
        anuncio_info_aux['endereco'] = soup.findAll('a', {'class': 'anchor'})[0].find('p').text
    except:
        anuncio_info_aux['endereco'] = np.nan

    try:
        #detalhes
        anuncio_info_aux['detalhes'] = soup.findAll('div',{'class': 'list'})[0].text.split()
    except:
        anuncio_info_aux['detalhes'] = np.nan

    try:
        # aluguel
        anuncio_info_aux['aluguel'] = soup.findAll('div',{'class': 'rental'})[0].find('h3').text
    except:
        anuncio_info_aux['aluguel'] = np.nan

    try:
        # condominio
        anuncio_info_aux['condominio'] = soup.findAll('div',{'class': 'condominium'})[0].findAll('dl')[0].text
    except:
        anuncio_info_aux['condominio'] = np.nan

    try:
        # iptu
        anuncio_info_aux['iptu'] = soup.findAll('div',{'class': 'condominium'})[0].findAll('dl')[1].text
    except:
        anuncio_info_aux['iptu'] = np.nan

    try:
        # características do imóvel
        anuncio_info_aux['catacteristicas_imovel'] = ' '.join([i.text for i in soup.findAll('article',{'class':'col-md-7 col-lg-8 details-property'})[0].findAll('p')[0]])
    except:
        anuncio_info_aux['catacteristicas_imovel'] = np.nan
    
    try:
        # detalhes do condominio
        anuncio_info_aux['detalhes_condominio'] = ', '.join([i.text for i in soup.findAll('article',{'class':'col-md-7 col-lg-8 details-property'})[0].findAll('p')[1::]])
    except:
        anuncio_info_aux['detalhes_condominio'] = np.nan

    try:
        # mais detalhes do imóvel
        anuncio_info_aux['mais_detalhes_imovel'] = ', '.join([i.text.replace('\n', '').replace('  ', '') for i in soup.findAll('ul',{'class':'list-arrow'})[0]])
    except:
        anuncio_info_aux['mais_detalhes_imovel'] = np.nan
        

    return anuncio_info_aux

def get_infos_anuncios(anuncios_links):
    anuncios_infos = []
    for link in anuncios_links[0].tolist():

        print(link)        
        options = webdriver.ChromeOptions()
        # options.add_experimental_option("prefs", {"profile.managed_default_content_settings.images": 2,})
        # options.add_argument('--headless') 

        driver = webdriver.Chrome(options=options)
        anuncio_info_aux = get_info_anuncio(driver, link)

        anuncios_infos.append(anuncio_info_aux)

    return pd.DataFrame(anuncios_infos)

In [5]:
LINK = 'https://cilar.com.br/alugar/apartamento/curitiba-pr?valueType=price&valueMin=100.00&valueMax=30000.00&areaType=area_total&areaMin=0&areaMax=10000&order=1&page=1'

last_page = get_last_page(LINK)

anuncios_links = get_anuncios_links(last_page)

anuncios = get_infos_anuncios(anuncios_links)

https://cilar.com.br/alugar/apartamento-no-6-andar-com-3-quartos-batel-curitiba/id/81490196/code/02433.024


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=132.0.6834.159)
Stacktrace:
#0 0x56c33466e53a <unknown>
#1 0x56c334169f00 <unknown>
#2 0x56c33413fb6f <unknown>
#3 0x56c3341e695d <unknown>
#4 0x56c3341fc945 <unknown>
#5 0x56c3341de343 <unknown>
#6 0x56c3341ab78a <unknown>
#7 0x56c3341ac9de <unknown>
#8 0x56c3346382cb <unknown>
#9 0x56c33463c242 <unknown>
#10 0x56c3346257ac <unknown>
#11 0x56c33463cdf7 <unknown>
#12 0x56c334609b2f <unknown>
#13 0x56c33465d1a8 <unknown>
#14 0x56c33465d370 <unknown>
#15 0x56c33466d3b6 <unknown>
#16 0x7b044ec9caa4 <unknown>
#17 0x7b044ed29c3c <unknown>


In [10]:
anuncios.to_csv('anuncios-cilar.csv', index=False)

In [11]:
anuncios

,titulo,endereco,detalhes,aluguel,condominio,iptu,catacteristicas_imovel,detalhes_condominio,mais_detalhes_imovel
0,Apartamento no 15º andar com 3 quartos,"Rua Professor Pedro Viriato Parigot Souza, 306...","[Características, do, imóvel, Área, Total, 421...","AluguelR$ 13.000,00","Condominio R$ 3.189,70","IPTU R$ 838,00",1ª LOCAÇÃO Apartamento de alto padrão contend...,"Piscina, Academia, Salão de festas, Playground...","Andar: 15, Ano de construção: 2021, Aptos por ..."
1,Apartamento no 1º andar com 3 quartos,"Avenida Paraná, 111 - Cabral - Curitiba","[Características, do, imóvel, Área, Total, 403...","AluguelR$ 12.000,00","Condominio R$ 2.348,64","IPTU R$ 782,96",Apartamento mobiliado contendo: 1 suíte com cl...,"Elevador, Garagem Coberta","Andar: 01, Ano de construção: 1999, Aptos por ..."
2,Apartamento no 1º andar com 4 quartos,"Rua Deputado Joaquim José Pedrosa, 618 - Cabra...","[Características, do, imóvel, Área, Total, 316...","AluguelR$ 5.000,00","Condominio R$ 1.926,04","IPTU R$ 404,97",3 salas com teto rebaixado sendo 1 com sacada ...,"Salão de festas, Playground, Quadra de esporte...","Água: sim, Andar: 01, Ano de construção: 1979,..."
3,Apartamento no 11º andar com 4 quartos,"Avenida Sete de Setembro, 4699 - Batel - Curitiba","[Características, do, imóvel, Área, Total, 227...","AluguelR$ 4.000,00","Condominio R$ 1.350,00","IPTU R$ 428,00",1 sala para 2 ambientes; 1 sala de jantar; 1 l...,"Academia, Salão de festas, Churrasqueira, Quad...","Andar: 11, Ano de construção: 1989, Aptos por ..."
4,Apartamento com 3 quartos,"Avenida República Argentina, 1690 - Portão - C...","[Características, do, imóvel, Área, Total, 169...","AluguelR$ 3.900,00","Condominio R$ 804,60","IPTU R$ 158,00","1 cozinha com armário, pia tampo granito, 1 me...",Garagem Coberta,"Ano de construção: 1986, Aptos por andar: 4, Á..."
...,...,...,...,...,...,...,...,...,...
455,Apartamento no 2º andar com 1 quarto,"Rua Lamenha Lins, 1486 - Centro - Curitiba","[Características, do, imóvel, Área, Total, 0,0...","AluguelR$ 1.100,00","Condominio R$ 372,00",NaN,Apartamento no Rebouças em Curitiba. Nas imedi...,Jardim,"Andar: 2, Aquecimento: sim, Área de serviço: 1..."
456,Apartamento com 1 quarto,"R Amintas de Barros, 164 - Centro - Curitiba","[Características, do, imóvel, Área, Total, 25,...","AluguelR$ 1.080,00","Condominio R$ 400,00",NaN,Imediações: TEATRO GUAÍRA Detalhes: STUDIO M...,Elevador,"Ano de construção: 23 Ano(s), Área privativa: ..."
457,Apartamento com 2 quartos,"Rua Desembargador Otávio do Amaral, 717 - Bigo...","[Características, do, imóvel, Área, Total, 110...","AluguelR$ 1.000,00","Condominio R$ 1.600,00","IPTU R$ 100,00",Apartamento Bigorrilho. Venha conhecer e se ...,Garagem Coberta,"Área privativa: 70, Área total: 110, BWC Socia..."
458,Apartamento no FAZENDINHA,"R ANTONIO DALMARCO, 400 - FAZENDINHA - Curitiba","[Características, do, imóvel, Área, Total, 55,...","AluguelR$ 1.000,00","Condominio R$ 354,00","IPTU R$ 25,32",Pontos de Referência: Bosque Fazendinha e Supe...,,"Área privativa: 44.87, Área total: 55.41, Edif..."
